In [ ]:
# Importing necessary libraries
import time
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, TensorDataset
import torchvision.transforms as transforms


Load synthetic data

In [ ]:
syn_images = np.loadtxt('Image_CVAE/images.txt')
syn_labels = np.loadtxt('Image_CVAE/labels.txt')
syn_angles = np.loadtxt('Image_CVAE/angles.txt')

In [ ]:
# Data preprocessing
n_syn = 40000  # Number of synthetic samples
syn_data = np.reshape(syn_images, (n_syn, 90, 90))  # Reshape images to 90x90
syn_data = syn_data[:, np.newaxis, :, :]  # Add a new axis to match model input requirements
syn_angles_std = syn_angles / 90  # Normalize angles by dividing by 90

In [ ]:
# Convert data to PyTorch tensors
syn_images_tensor = torch.tensor(syn_data, dtype=torch.float32)
syn_labels_tensor = torch.tensor(syn_labels, dtype=torch.float32)
syn_angles_tensor = torch.tensor(syn_angles_std, dtype=torch.float32)

# Split data into training and testing sets
X_train, X_test, y1_train, y1_test, y2_train, y2_test = train_test_split(
    syn_images_tensor, syn_labels_tensor, syn_angles_tensor, test_size=0.05, random_state=42
)

# Create TensorDataset
train_dataset = TensorDataset(X_train, y1_train, y2_train)
test_dataset = TensorDataset(X_test, y1_test, y2_test)

# Create DataLoader
batch_size = 512  
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
# Device
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print('Device:', device)

CNN

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        
        # Define convolutional layers
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)  # First convolutional layer
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)  # Second convolutional layer
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)  # Third convolutional layer

        # Define fully connected layers
        self.fc1 = nn.Linear(128 * 11 * 11, 128)  # Fully connected layer 1
        self.fc2 = nn.Linear(128, 1)  # Fully connected layer 2 for first output
        self.fc3 = nn.Linear(128, 1)  # Fully connected layer 3 for second output

    def forward(self, x):
        # Apply convolutional layers with ReLU activation and max pooling
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, 2)
        
        # Flatten the tensor
        x = x.view(-1, 128 * 11 * 11)
        
        # Apply fully connected layers with activation functions
        x = F.relu(self.fc1(x))
        x1 = torch.sigmoid(self.fc2(x))  # Output 1 with sigmoid activation
        x2 = torch.tanh(self.fc3(x))  # Output 2 with tanh activation
        
        return x1, x2


In [ ]:
# Initialize the model and move it to the specified device
model = SimpleCNN()
model = model.to(device)


In [ ]:
# Define loss functions
criterion_y1 = nn.BCELoss()  # Binary Cross Entropy Loss for Linearity
criterion_y2 = nn.MSELoss()  # Mean Squared Error Loss for Orientation

In [ ]:
# Initialize optimizer and learning rate scheduler
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.995)

Network training

In [ ]:
# Training loop
start_time = time.time()
train_losses_y1 = [] 
train_losses_y2 = [] 
train_losses = [] 
Accuracy_test = []
test_losses_y1 = [] 
test_losses_y2 = [] 
test_losses = [] 

epochs = 2
for epoch in range(epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    running_loss_y1 = 0.0
    running_loss_y2 = 0.0
    for images, labels, angles in train_loader:
        labels = labels.view(-1, 1).to(device).float()
        angles = angles.view(-1, 1).to(device).float()
        
        optimizer.zero_grad() 
        outputs_y1, outputs_y2 = model(images.to(device))

        loss_y1 = criterion_y1(outputs_y1, labels)
        loss_y2 = criterion_y2(outputs_y2, angles)
        loss = loss_y1 + loss_y2
        
        loss.backward()
        optimizer.step()
        running_loss_y1 += loss_y1.item()
        running_loss_y2 += loss_y2.item()
        running_loss += loss.item()
    
    mean_loss = running_loss / len(train_loader)
    mean_loss_y1 = running_loss_y1 / len(train_loader)
    mean_loss_y2 = running_loss_y2 / len(train_loader)
    train_losses_y1.append(mean_loss_y1)
    train_losses_y2.append(mean_loss_y2)
    train_losses.append(mean_loss)
    scheduler.step()
    
    # Evaluation on test set
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        X_test = X_test.to(device)
        y1_test = y1_test.to(device).float().view(-1, 1)
        y2_test = y2_test.to(device).float().view(-1, 1)
        
        outputs_y1, outputs_y2 = model(X_test)
        
        loss_y1 = criterion_y1(outputs_y1, y1_test)
        loss_y2 = criterion_y2(outputs_y2, y2_test)
        loss = loss_y1 + loss_y2
        
        predicted = (outputs_y1 > 0.5).float()  # Convert output to 0 or 1
        total_t = y1_test.size(0)
        correct_t = (predicted.view(-1) == y1_test).sum().item()
    
    test_losses_y1.append(loss_y1)
    test_losses_y2.append(loss_y2)
    test_losses.append(loss)
    Accuracy_test.append(correct_t / total_t)
    
    print(f"Epoch {epoch+1}, Loss: {mean_loss:.4f}, Loss_y1: {mean_loss_y1:.4f}, Loss_y2: {mean_loss_y2:.4f}, "
          f"Test Loss: {loss:.4f}, Test Loss_y1: {loss_y1:.4f}, Test Loss_y2: {loss_y2:.4f}, "
          f"Test Accuracy: {100 * correct_t / total_t:.2f}%")

end_time = time.time()
print('Training time: ', end_time - start_time, ' s')


Save network

In [ ]:
# torch.save(model, 'CharacterizationNet.pth')

Loss curves

In [ ]:
# Training Loss
plt.figure(figsize=(12, 5))
plt.subplot(1, 3, 1)
plt.plot(train_losses_y1, label='Training Loss y1')
plt.plot(train_losses_y2, label='Training Loss y2')
plt.plot(train_losses, label='Training Loss')
plt.yscale('log')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.legend()

# Test Loss
plt.subplot(1, 3, 2)
plt.plot(test_losses_y1, label='Test Loss y1')
plt.plot(test_losses_y2, label='Test Loss y2')
plt.plot(test_losses, label='Test Loss')
plt.yscale('log')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Test Loss')
plt.legend()

# Test accuracy
plt.subplot(1, 3, 3)
plt.plot(Accuracy_test, label='Test Accuracy', color='orange')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Test Accuracy')
plt.legend()

